In [13]:
import pandas as pd
import os


# Start timer

# get current directory
path = os.getcwd()
print("Current Directory", path)
 
# prints parent directory
parent_dir = os.path.abspath(os.path.join(path, os.pardir))
input_dir = f"{parent_dir}/data"

Current Directory /Users/NachoCorcuera/PycharmProjects/Pandas-Polars-PySpark-BenchMark/notebooks


# Pandas

In [3]:
import pandas as pd
import time


def join_df():
    regions = [
        "Europe", 
        "North America", 
        "Asia", 
        "Sub-Saharan Africa", 
        "Central America and the Caribbean", 
        "Middle East and North Africa", 
        "Australia and Oceania"
    ]

    aliases = [
        "EU", 
        "NA", 
        "AS", 
        "SSA", 
        "CA", 
        "MENA", 
        "AUS"
    ]

    # Create the DataFrame using Pandas
    df_regions = pd.DataFrame({
        "Region": regions,
        "Alias": aliases
    })

    # Display the DataFrame to verify its contents
    return df_regions

def processing_df(file):
    times = {}
    start_time_full = time.time()

    # Reading time
    df = pd.read_csv(f"{input_dir}/{file}.csv")
    end_time = time.time()
    times["read_csv"] = end_time - start_time_full

    # Filtering time
    start_time = time.time()
    filtered_pandas = df[df['Total Profit'] > 2000]
    end_time = time.time()
    times["filter"] = end_time - start_time

    # Aggregation
    start_time = time.time()
    aggregated_pandas = df.groupby('Region').agg(
        sales=('Total Profit', 'sum'),
        sales_mean=('Total Profit', 'mean'),
        sales_max=('Total Profit', 'max'),
        sales_min=('Total Profit', 'min'),
        sales_median=('Total Profit', 'median')
    )
    end_time = time.time()
    times["aggregation"] = end_time - start_time

    # Joining time
    start_time = time.time()
    df_regions = join_df()  # Assuming join_df returns a DataFrame to join
    df_joined = df.merge(df_regions, on="Region", how="left")
    end_time = time.time()
    times["join"] = end_time - start_time

    # Writing time
    start_time = time.time()
    df_joined.to_csv("testing_write_2.csv", index=False)
    end_time = time.time()
    times["write"] = end_time - start_time

    return times

In [6]:
file_list = ["sales_50000",
             "sales_250000",
             "sales_1000000",
             "sales_5000000",
             "sales_25000000"]


times = {file: processing_df(file) for file in file_list}

In [7]:
times

{'sales_50000': {'read_csv': 0.060051918029785156,
  'filter': 0.004604339599609375,
  'aggregation': 0.009542226791381836,
  'join': 0.006765127182006836,
  'write': 0.28705716133117676},
 'sales_250000': {'read_csv': 0.2204129695892334,
  'filter': 0.011040687561035156,
  'aggregation': 0.01796126365661621,
  'join': 0.02841019630432129,
  'write': 0.9712498188018799},
 'sales_1000000': {'read_csv': 0.8394689559936523,
  'filter': 0.05398988723754883,
  'aggregation': 0.06402993202209473,
  'join': 0.11523294448852539,
  'write': 3.826240062713623},
 'sales_5000000': {'read_csv': 4.053663969039917,
  'filter': 0.24633097648620605,
  'aggregation': 0.30512070655822754,
  'join': 0.5451042652130127,
  'write': 19.169981956481934},
 'sales_25000000': {'read_csv': 27.962117195129395,
  'filter': 2.4532999992370605,
  'aggregation': 2.1596198081970215,
  'join': 3.90976619720459,
  'write': 151.89875292778015}}

In [14]:
from pathlib import Path

def processing_parquet_df(file):
    times = {}
    start_time_full = time.time()

    # Reading time
    data_dir = Path(f"{input_dir}/{file}")

    df = pd.concat(
    pd.read_parquet(parquet_file, engine='pyarrow')
    for parquet_file in data_dir.glob('*.parquet')
    )

    end_time = time.time()
    times["read_csv"] = end_time - start_time_full

    # Filtering time
    start_time = time.time()
    filtered_pandas = df[df['Total Profit'] > 2000]
    end_time = time.time()
    times["filter"] = end_time - start_time

    # Aggregation
    start_time = time.time()
    aggregated_pandas = df.groupby('Region').agg(
        sales=('Total Profit', 'sum'),
        sales_mean=('Total Profit', 'mean'),
        sales_max=('Total Profit', 'max'),
        sales_min=('Total Profit', 'min'),
        sales_median=('Total Profit', 'median')
    )
    end_time = time.time()
    times["aggregation"] = end_time - start_time

    # Joining time
    start_time = time.time()
    df_regions = join_df()  # Assuming join_df returns a DataFrame to join
    df_joined = df.merge(df_regions, on="Region", how="left")
    end_time = time.time()
    times["join"] = end_time - start_time

    # Writing time
    start_time = time.time()
    df_joined.to_parquet("testing_write_3.parquet", index=False)
    end_time = time.time()
    times["write"] = end_time - start_time

    return times

In [15]:
file = "parquet/sales_25000000"
processing_parquet_df(file)

{'read_csv': 5.053792953491211,
 'filter': 1.944251298904419,
 'aggregation': 1.9538791179656982,
 'join': 5.851027965545654,
 'write': 15.206745147705078}

In [10]:
data_dir = Path(file)
for parquet_file in data_dir.glob('*.parquet'):
    print(parquet_file)